In [ ]:
##ここで、このプログラムが使うパッケージを読み込んでいます。
import numpy as np
 
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
import cv2

##ここで、手書き数字の有名なパッケージmnistを読み込んでいます。60000,10000は、それぞれ訓練データ、テストデータの数。28は（縦横の）画像サイズです。
##このmnistを用いて、数字認識のモデルを作ります。そのとき「訓練データで訓練→テストデータで訓練の精度を確かめる」を繰り返して、モデルをよくします。
(x_train0, y_train0), (x_test0, y_test0) = mnist.load_data()
print(x_train0.shape)
print(y_train0.shape)
print(x_test0.shape)
print(y_test0.shape)

In [ ]:
#mnist画像を最初の5つを、答えとセットで表示
for i in range(5):
    print("答えは", y_train0[i])
    plt.imshow(x_train0[i,:,:])
    plt.gray()
    plt.show()

In [ ]:
##データの変換です。
##ここで、mnistデータを増やしています。trainもtestも、少し拡大したデータを、付け加えました。

w=2
dsize = len(x_train0)
x=np.array([cv2.resize(x_train0[i,:,:], (28+2*w,28+2*w))[w:28+w,w:28+w] for i in range(dsize)])
X_train=np.array([x_train0, x]).reshape(2*dsize,28,28,1).astype('float32') / 255
y_train=np.array([y_train0, y_train0]).reshape(2*dsize,)

dsize = len(x_test0)
x=np.array([cv2.resize(x_test0[i,:,:], (28+2*w,28+2*w))[w:28+w,w:28+w] for i in range(dsize)])
X_test=np.array([x_test0, x]).reshape(2*dsize,28,28,1).astype('float32') / 255
y_test=np.array([y_test0, y_test0]).reshape(2*dsize,)

##ここはデータの形式の変換
nb_classes = 10
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test  = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
print(X_train.shape)
#mnist画像を最初の5つを、拡大前と拡大後で比較。また、y_trainのデータ形式が変換されているのも分かります。
#画像はわずかな違いですが、この拡大後のデータが増えるだけで精度はぐっと良くなります。
#本当は、上下左右に少しずらした画像を付け加えても精度はよくなりますが、増やしすぎると計算時間がかかるので断念。
for i in range(5):
    print("答えは", y_train[i])
    plt.subplot(121)
    plt.imshow(X_train[i,:,:,0])
    plt.gray()
    plt.subplot(122)
    plt.imshow(X_train[60000+i,:,:,0])
    plt.gray()
    plt.show()

In [ ]:
#ここが、モデルの学習です（時間がかかります。普通は10-30分）。これは
#以下の操作を最大10回(nb_epoch)行って
###訓練データを9:1に分けて(validation_split)
###9割の方で学習し（その途中経過が"acc: ***"の数字です。acc=accuracy=正答率）
###1割で訓練結果を確かめる（"val_acc:***"の数字）
#10回終わるか、val_accが悪くなった終了し、テストデータで正答率を確かめる（結果が"Test accuracy:***"））

# 画像を何個ずつでまとめて処理するか
batch_size = 1024
# 最大、何回の処理を行うか
nb_epoch   = 10

##このあたりの設定は、いくつにするかは難しい
# 畳み込みフィルタの数
#nb_filters = 24
# 最大プーリングのための pooling area のサイズ
nb_pool = 2
# 畳み込みカーネルサイズ
#nb_conv = 7

model = Sequential()
model.add(Conv2D(16, (5, 5), input_shape=(28, 28, 1)))
model.add(Activation('relu'))
model.add(Conv2D(5, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

#execute
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=["accuracy"])
hist = model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
          verbose=1, validation_data=(X_test, y_test),
                 callbacks=[early_stopping], validation_split=0.1)
 

# evaluate
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# plot loss
loss = hist.history['loss']
val_loss = hist.history['val_loss']

nb_epoch = len(loss)
plt.plot(range(nb_epoch), loss, marker='.', label='loss')
plt.plot(range(nb_epoch), val_loss, marker='.', label='val_loss')
plt.legend(loc='best', fontsize=10)
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
#テスト結果をいくつか表示してみます
##ためしに4の判定結果を見てみます（letter=5とかにすれば、5の判定結果が出ます）
letter = 4
np.set_printoptions(suppress=True, precision=10)

for i in range(100):
    if y_test[i,letter] == 1.0:
        y = model.predict(X_test[i,:,:,0].reshape(1,28,28,1))
        print("モデルが判定した結果は", np.argmax(y))
        print("参考：モデルが0,1,2,...のそれぞれにどのくらい一致すると判断したかのデータ")
        print(y)
        plt.imshow(X_test[i,:,:,0].reshape((28,28)))
        plt.gray()
        plt.show()
        print("------")

In [ ]:
#取り込んだ画像のコントラストを、モデルに合わせて調整するプログラム
import math
def contrast(image, a, b):
  lut = [ max(0, np.uint8(255.0 / (1 + math.exp(-a * (i - b) / 255.)))-b) for i in range(256)] 
  result_image = np.array( [ lut[value] for value in image.flat], dtype=np.uint8 )
  result_image = result_image.reshape(image.shape)
  return result_image

In [ ]:
import os
os.getcwd()

In [ ]:
###ここで画像ファイルを指定
filename = "f.png"

###以下、画像を読み込んで、モデルが判断して、最後に画像と結果を出力しています。
temp = cv2.imread(filename)
imggray = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
imggray = cv2.resize(imggray, (28,28))
print("左：28*28にした画像、右：コントラストを調整後（これで学習）")
plt.subplot(121)
plt.imshow(255-imggray)
plt.gray()
imggray2 = contrast(255-imggray, 30,40).astype("float32")
#print(imggray2)
plt.subplot(122)
plt.imshow(imggray2)
plt.gray()
plt.show()
y = model.predict(imggray2.reshape(1,28,28,1)/255)
print("モデルが判定した結果は", np.argmax(y))
print("モデルが0,1,2,...のそれぞれにどのくらい一致すると判断したかのデータ")
print(y)
print("------")